# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [1]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# magic word for producing visualizations in notebook
%matplotlib inline

In [48]:
#useful tool for debugging
!pip install icecream

In [49]:
#all other imports 
import icecream as ic

In [2]:
#useful variables
addtional_customer_cols = ['PRODUCT_GROUP', 'CUSTOMER_GROUP', 'ONLINE_PURCHASE']
azdias_num_rows = 891221
azdias_num_cols = 366
customers_num_rows = 191652
customers_num_cols = 369

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

In [3]:
# load in the data
azdias = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_AZDIAS_052018.csv', sep=';')
customers = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_CUSTOMERS_052018.csv', sep=';')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#observe warning above - 5 columns have mixed types.  Then need to be identified and addressed

In [4]:
#check the correct rows and cols are loaded
print("azdias: ",azdias.shape)
print("customers: ",customers.shape)

assert azdias.shape[0] == 891221, "azdias row count incorrect"
assert azdias.shape[1] == 366, "azdias col count incorrect"
assert customers.shape[0] == 191652, "azdias row count incorrect"
assert customers.shape[1] == 369, "azdias col count incorrect"

#make a copy of the df
azdias_org = azdias
customers_org = customers

azdias:  (891221, 366)
customers:  (191652, 369)


In [33]:
#load the values and attributes
attributes_value_ranges = pd.read_excel('DIAS Attributes - Values 2017.xlsx', header =1)
attributes_descriptions = pd.read_excel('DIAS Information Levels - Attributes 2017.xlsx', header = 1)

In [6]:
azdias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891221 entries, 0 to 891220
Columns: 366 entries, LNR to ALTERSKATEGORIE_GROB
dtypes: float64(267), int64(93), object(6)
memory usage: 2.4+ GB


In [7]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191652 entries, 0 to 191651
Columns: 369 entries, LNR to ALTERSKATEGORIE_GROB
dtypes: float64(267), int64(94), object(8)
memory usage: 539.5+ MB


In [5]:
azdias.to_csv('azdias.csv', index=False, sep=';')

In [5]:
customers.to_csv('customers.csv', index=False, sep=';')

In [8]:
pd.set_option('display.max_columns', None)
azdias.head(5)

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,ANZ_HH_TITEL,ANZ_KINDER,ANZ_PERSONEN,ANZ_STATISTISCHE_HAUSHALTE,ANZ_TITEL,ARBEIT,BALLRAUM,CAMEO_DEU_2015,CAMEO_DEUG_2015,CAMEO_INTL_2015,CJT_GESAMTTYP,CJT_KATALOGNUTZER,CJT_TYP_1,CJT_TYP_2,CJT_TYP_3,CJT_TYP_4,CJT_TYP_5,CJT_TYP_6,D19_BANKEN_ANZ_12,D19_BANKEN_ANZ_24,D19_BANKEN_DATUM,D19_BANKEN_DIREKT,D19_BANKEN_GROSS,D19_BANKEN_LOKAL,D19_BANKEN_OFFLINE_DATUM,D19_BANKEN_ONLINE_DATUM,D19_BANKEN_ONLINE_QUOTE_12,D19_BANKEN_REST,D19_BEKLEIDUNG_GEH,D19_BEKLEIDUNG_REST,D19_BILDUNG,D19_BIO_OEKO,D19_BUCH_CD,D19_DIGIT_SERV,D19_DROGERIEARTIKEL,D19_ENERGIE,D19_FREIZEIT,D19_GARTEN,D19_GESAMT_ANZ_12,D19_GESAMT_ANZ_24,D19_GESAMT_DATUM,D19_GESAMT_OFFLINE_DATUM,D19_GESAMT_ONLINE_DATUM,D19_GESAMT_ONLINE_QUOTE_12,D19_HANDWERK,D19_HAUS_DEKO,D19_KINDERARTIKEL,D19_KONSUMTYP,D19_KONSUMTYP_MAX,D19_KOSMETIK,D19_LEBENSMITTEL,D19_LETZTER_KAUF_BRANCHE,D19_LOTTO,D19_NAHRUNGSERGAENZUNG,D19_RATGEBER,D19_REISEN,D19_SAMMELARTIKEL,D19_SCHUHE,D19_SONSTIGE,D19_SOZIALES,D19_TECHNIK,D19_TELKO_ANZ_12,D19_TELKO_ANZ_24,D19_TELKO_DATUM,D19_TELKO_MOBILE,D19_TELKO_OFFLINE_DATUM,D19_TELKO_ONLINE_DATUM,D19_TELKO_ONLINE_QUOTE_12,D19_TELKO_REST,D19_TIERARTIKEL,D19_VERSAND_ANZ_12,D19_VERSAND_ANZ_24,D19_VERSAND_DATUM,D19_VERSAND_OFFLINE_DATUM,D19_VERSAND_ONLINE_DATUM,D19_VERSAND_ONLINE_QUOTE_12,D19_VERSAND_REST,D19_VERSI_ANZ_12,D19_VERSI_ANZ_24,D19_VERSI_DATUM,D19_VERSI_OFFLINE_DATUM,D19_VERSI_ONLINE_DATUM,D19_VERSI_ONLINE_QUOTE_12,D19_VERSICHERUNGEN,D19_VOLLSORTIMENT,D19_WEIN_FEINKOST,DSL_FLAG,EINGEFUEGT_AM,EINGEZOGENAM_HH_JAHR,EWDICHTE,EXTSEL992,FINANZ_ANLEGER,FINANZ_HAUSBAUER,FINANZ_MINIMALIST,FINANZ_SPARER,FINANZ_UNAUFFAELLIGER,FINANZ_VORSORGER,FINANZTYP,FIRMENDICHTE,GEBAEUDETYP,GEBAEUDETYP_RASTER,GEBURTSJAHR,GEMEINDETYP,GFK_URLAUBERTYP,GREEN_AVANTGARDE,HEALTH_TYP,HH_DELTA_FLAG,HH_EINKOMMEN_SCORE,INNENSTADT,KBA05_ALTER1,KBA05_ALTER2,KBA05_ALTER3,KBA05_ALTER4,KBA05_ANHANG,KBA05_ANTG1,KBA05_ANTG2,KBA05_ANTG3,KBA05_ANTG4,KBA05_AUTOQUOT,KBA05_BAUMAX,KBA05_CCM1,KBA05_CCM2,KBA05_CCM3,KBA05_CCM4,KBA05_DIESEL,KBA05_FRAU,KBA05_GBZ,KBA05_HERST1,KBA05_HERST2,KBA05_HERST3,KBA05_HERST4,KBA05_HERST5,KBA05_HERSTTEMP,KBA05_KRSAQUOT,KBA05_KRSHERST1,KBA05_KRSHERST2,KBA05_KRSHERST3,KBA05_KRSKLEIN,KBA05_KRSOBER,KBA05_KRSVAN,KBA05_KRSZUL,KBA05_KW1,KBA05_KW2,KBA05_KW3,KBA05_MAXAH,KBA05_MAXBJ,KBA05_MAXHERST,KBA05_MAXSEG,KBA05_MAXVORB,KBA05_MOD1,KBA05_MOD2,KBA05_MOD3,KBA05_MOD4,KBA05_MOD8,KBA05_MODTEMP,KBA05_MOTOR,KBA05_MOTRAD,KBA05_SEG1,KBA05_SEG10,KBA05_SEG2,KBA05_SEG3,KBA05_SEG4,KBA05_SEG5,KBA05_SEG6,KBA05_SEG7,KBA05_SEG8,KBA05_SEG9,KBA05_VORB0,KBA05_VORB1,KBA05_VORB2,KBA05_ZUL1,KBA05_ZUL2,KBA05_ZUL3,KBA05_ZUL4,KBA13_ALTERHALTER_30,KBA13_ALTERHALTER_45,KBA13_ALTERHALTER_60,KBA13_ALTERHALTER_61,KBA13_ANTG1,KBA13_ANTG2,KBA13_ANTG3,KBA13_ANTG4,KBA13_ANZAHL_PKW,KBA13_AUDI,KBA13_AUTOQUOTE,KBA13_BAUMAX,KBA13_BJ_1999,KBA13_BJ_2000,KBA13_BJ_2004,KBA13_BJ_2006,KBA13_BJ_2008,KBA13_BJ_2009,KBA13_BMW,KBA13_CCM_0_1400,KBA13_CCM_1000,KBA13_CCM_1200,KBA13_CCM_1400,KBA13_CCM_1401_2500,KBA13_CCM_1500,KBA13_CCM_1600,KBA13_CCM_1800,KBA13_CCM_2000,KBA13_CCM_2500,KBA13_CCM_2501,KBA13_CCM_3000,KBA13_CCM_3001,KBA13_FAB_ASIEN,KBA13_FAB_SONSTIGE,KBA13_FIAT,KBA13_FORD,KBA13_GBZ,KBA13_HALTER_20,KBA13_HALTER_25,KBA13_HALTER_30,KBA13_HALTER_35,KBA13_HALTER_40,KBA13_HALTER_45,KBA13_HALTER_50,KBA13_HALTER_55,KBA13_HALTER_60,KBA13_HALTER_65,KBA13_HALTER_66,KBA13_HERST_ASIEN,KBA13_HERST_AUDI_VW,KBA13_HERST_BMW_BENZ,KBA13_HERST_EUROPA,KBA13_HERST_FORD_OPEL,KBA13_HERST_SONST,KBA13_HHZ,KBA13_KMH_0_140,KBA13_KMH_110,KBA13_KMH_140,KBA13_KMH_140_210,KBA13_KMH_180,KBA13_KMH_210,KBA13_KMH_211,KBA13_KMH_250,KBA13_KMH_251,KBA13_KRSAQUOT,KBA13_KRSHERST_AUDI_VW,KBA13_KRSHERST_BMW_BENZ,KBA13_KRSHERST_FORD_OPEL,KBA13_KRSSEG_KLEIN,KBA13_KRSSEG_OBER,KBA13_KRSSEG_VAN,KBA13_KRSZUL_NEU,KBA13_KW_0_60,KBA13_KW_110,KBA13_KW_120,KBA13_KW_121,KBA13_KW_30,KBA13_KW_40,KBA13_KW_50,KBA13_KW_60,KBA13_KW_61_120,KBA13_KW_70,KBA13_KW_80,

In [ ]:
customers.head(5)

In [ ]:
Initial Observations:
    
Warning on read_csv: Columns (18,19) have mixed types

LNR: first column: is it an index or identity? 

D19_LETZTER_KAUF_BRANCHE looks like it has other col names listed: check this maybe encode?
    
EINGEFUEGT_AM looks like a datetime object; no date times are listed in the info()

MIN_GEBAEUDEJAHR: looks like a year (jahr means year!): should be int not float
        
OST_WEST_KZ: has type object: can be encoded?

In [12]:
#look at non-numeric attributes #1
non_num_attributes_customers = customers.select_dtypes(include='object')
non_num_attributes_azdias = azdias.select_dtypes(include='object')


In [14]:
non_num_attributes_azdias.describe()

,CAMEO_DEU_2015,CAMEO_DEUG_2015,CAMEO_INTL_2015,D19_LETZTER_KAUF_BRANCHE,EINGEFUEGT_AM,OST_WEST_KZ
count,792242,792242,792242,634108,798073,798073
unique,45,19,43,35,5162,2
top,6B,8,51,D19_UNBEKANNT,1992-02-10 00:00:00,W
freq,56672,78023,77576,195338,383738,629528


In [ ]:
#look at non-numeric attributes #2
non_num_attributes_azdias.head
#non_num_attributes.describe()

In [15]:
non_num_attributes_azdias.isna().sum().sort_values(ascending=False).head()


CAMEO_INTL_2015    50428
CAMEO_DEUG_2015    50428
CAMEO_DEU_2015     50428
OST_WEST_KZ        49927
EINGEFUEGT_AM      49927
dtype: int64

In [16]:
non_num_attributes_customers.isna().sum().sort_values(ascending=False).head()

CAMEO_INTL_2015    50428
CAMEO_DEUG_2015    50428
CAMEO_DEU_2015     50428
OST_WEST_KZ        49927
EINGEFUEGT_AM      49927
dtype: int64

In [ ]:
NB the same columns are listed

In [23]:
#there are 891221 rows on the azdias df
len(pd.unique(azdias['LNR']))

891221

In [24]:
#there are 191652 rows on the customers df
len(pd.unique(customers['LNR']))

191652

In [ ]:
LNR is and id column and should be dropped

In [26]:
azdias['EINGEFUEGT_AM']= pd.to_datetime(azdias['EINGEFUEGT_AM'])
 
# Check the format of 'Date' column
azdias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891221 entries, 0 to 891220
Columns: 366 entries, LNR to ALTERSKATEGORIE_GROB
dtypes: datetime64[ns](1), float64(267), int64(93), object(5)
memory usage: 2.4+ GB


In [28]:
customers['EINGEFUEGT_AM'].unique()

array(['1992-02-12 00:00:00', nan, '1992-02-10 00:00:00', ...,
       '2008-04-25 00:00:00', '2005-03-30 00:00:00', '2008-07-14 00:00:00'], dtype=object)

In [29]:
customers['EINGEFUEGT_AM']= pd.to_datetime(customers['EINGEFUEGT_AM'])
 
# Check the format of 'Date' column
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191652 entries, 0 to 191651
Columns: 369 entries, LNR to ALTERSKATEGORIE_GROB
dtypes: datetime64[ns](1), float64(267), int64(94), object(7)
memory usage: 539.5+ MB


In [ ]:
#search for gros and fein

In [ ]:
dias_values["Attribute"] = dias_values["Attribute"].ffill()
dias_values.head(10)

In [28]:
def printListDetails(l, desc='no description', counter=1, to_sort=False):
    '''
    '''
    
    if to_sort:
        new_list = [str(i) for i in l]
        l = sorted(new_list)
        
    print (counter, desc, ":", 'len:', len(l), ',', l, '\n')
    counter = counter + 1
    return counter

In [31]:
#Analyse columns
azdias_cols = azdias.columns
customers_cols = customers.columns
attributes_ranges_cols_list = attributes_value_ranges.Attribute
attributes_descriptions_cols_list = attributes_descriptions.Attribute

diff_customers_cols_versus_azdias_cols = list(set(customers_cols) - set(azdias_cols))
diff_azdias_cols_versus_customers_cols = list(set(azdias_cols) - set(customers_cols))
diff_attrib_ranges_cols_versus_customers_cols = list(set(attributes_ranges_cols_list) - set(customers_cols))
diff_customers_cols_versus_attrib_ranges_cols = list(set(customers_cols) - set(attributes_ranges_cols_list))
diff_attrib_ranges_cols_versus_azdias_cols = list(set(attributes_ranges_cols_list) - set(azdias_cols))
diff_azdias_cols_versus_attrib_ranges_cols = list(set(azdias_cols) - set(attributes_ranges_cols_list))
diff_between_missing_col_customers_azdias = list(set(diff_attrib_ranges_cols_versus_customers_cols) - set(diff_attrib_ranges_cols_versus_azdias_cols))
diff_between_missing_col_azdias_customers = list(set(diff_attrib_ranges_cols_versus_azdias_cols) - set(diff_attrib_ranges_cols_versus_customers_cols))
  
counter=1
counter=printListDetails(diff_customers_cols_versus_azdias_cols, "Customer versus azdias lists (expect 3 cols)", counter,to_sort=True)
counter=printListDetails(diff_azdias_cols_versus_customers_cols, "Azdias versus customers lists (expect 0 cols)", counter,to_sort=True)
counter=printListDetails(diff_attrib_ranges_cols_versus_customers_cols, "Attributes versus customers lists", counter,to_sort=True)
counter=printListDetails(diff_customers_cols_versus_attrib_ranges_cols, "Customers versus Attributes lists", counter,to_sort=True)
counter=printListDetails(diff_attrib_ranges_cols_versus_azdias_cols, "Attributes versus azdias lists", counter,to_sort=True)
counter=printListDetails(diff_azdias_cols_versus_attrib_ranges_cols, "Azdias versus Attributes lists", counter,to_sort=True)
counter=printListDetails(diff_between_missing_col_customers_azdias, "Excess missing cols customers versus azdias (expect 0 cols)", counter,to_sort=True)
counter=printListDetails(diff_between_missing_col_azdias_customers, "Excess missing cols azdias versus customers (expect 0 cols):", counter,to_sort=True)


1 Customer versus azdias lists (expect 3 cols) : len: 3 , ['CUSTOMER_GROUP', 'ONLINE_PURCHASE', 'PRODUCT_GROUP'] 

2 Azdias versus customers lists (expect 0 cols) : len: 0 , [] 

3 Attributes versus customers lists : len: 43 , ['BIP_FLAG', 'CAMEO_DEUINTL_2015', 'D19_BANKEN_DIREKT_RZ', 'D19_BANKEN_GROSS_RZ', 'D19_BANKEN_LOKAL_RZ', 'D19_BANKEN_REST_RZ', 'D19_BEKLEIDUNG_GEH_RZ', 'D19_BEKLEIDUNG_REST_RZ', 'D19_BILDUNG_RZ', 'D19_BIO_OEKO_RZ', 'D19_BUCH_RZ', 'D19_DIGIT_SERV_RZ', 'D19_DROGERIEARTIKEL_RZ', 'D19_ENERGIE_RZ', 'D19_FREIZEIT_RZ', 'D19_GARTEN_RZ', 'D19_HANDWERK_RZ', 'D19_HAUS_DEKO_RZ', 'D19_KINDERARTIKEL_RZ', 'D19_KK_KUNDENTYP', 'D19_KOSMETIK_RZ', 'D19_LEBENSMITTEL_RZ', 'D19_LOTTO_RZ', 'D19_NAHRUNGSERGAENZUNG_RZ', 'D19_RATGEBER_RZ', 'D19_REISEN_RZ', 'D19_SAMMELARTIKEL_RZ', 'D19_SCHUHE_RZ', 'D19_SONSTIGE_RZ', 'D19_TECHNIK_RZ', 'D19_TELKO_MOBILE_RZ', 'D19_TELKO_REST_RZ', 'D19_TIERARTIKEL_RZ', 'D19_VERSAND_REST_RZ', 'D19_VERSICHERUNGEN_RZ', 'D19_VOLLSORTIMENT_RZ', 'D19_WEIN_FEINKOST_R

In [25]:
print(diff_customers_cols_versus_azdias_cols)
diff_customers_cols_versus_azdias_cols=sorted(diff_customers_cols_versus_azdias_cols)
#print(sorted(diff_customers_cols_versus_azdias_cols))
print(diff_customers_cols_versus_azdias_cols)

['ONLINE_PURCHASE', 'CUSTOMER_GROUP', 'PRODUCT_GROUP']
['CUSTOMER_GROUP', 'ONLINE_PURCHASE', 'PRODUCT_GROUP']


In [ ]:
1. This is expected: we were told that the customer df has three extra columns
    
2. Expected: the azdias df does not have more cols than the customers df

3. Surprising: there are 43 more attribs listed than are used on the customers df

4. Surprising: there are 97 cols on the customers df that are not described in the attribs listed

5. Surprising: there are 43 more attribs listed than are used on the azdias df

6. Surprising: there are 94 cols on the azdias df that are not described in the attribs listed

7. Expected: the 'missing' cols are consistent 

8. Expected: the 'missing' cols are consistent 

In [9]:
#get the attributes that have unknown as a possible value
attributes_with_unknowns = attributes_value_ranges.loc[attributes_value_ranges['Meaning'] == 'unknown']


#attributes_with_unknowns.Value
#attributes_with_unknowns.Value.describe()
#titanic[titanic["Pclass"].isin([2, 3])]
#df = pd.DataFrame([-1,0,1,2,3])
#df.columns = ['val']
#df.iloc[0]['val']
#df
#df[df["val"].isin([2, 3])]
#x=df[df["val"].isin([2, 3])]
#print(x)

#attributes_with_unknowns.iloc[1][2]
#attributes_with_unknowns

#if df.iloc[0]['val'].isin(attributes_with_unknowns.iloc[1][2]):
#mList = [int(e) if e.isdigit() else e for e in mStr.split(',')]
#x = df.iloc[0].isin([int(e) if e.isdigit() else e for e in attributes_with_unknowns.iloc[1][2].split(',')])
#print(x)
#attributes_with_unknowns = attributes_value_ranges.loc[dias_values['Meaning'] == 'unknown' | dias_values['Meaning'] == 'no transactions known' | dias_values['Meaning'] == 'no transactions known']
attributes_with_unknowns = attributes_value_ranges.loc[attributes_value_ranges['Meaning'] == 'unknown']
attributes_with_unknowns = attributes_with_unknowns.append(attributes_value_ranges.loc[attributes_value_ranges['Meaning'] == 'no transactions known'])
attributes_with_unknowns = attributes_with_unknowns.append(attributes_value_ranges.loc[attributes_value_ranges['Meaning'] == 'no transaction known'])
attributes_with_unknowns.shape

(288, 4)

In [23]:
x = attributes_value_ranges.Meaning.unique()

unknown_terms = []
for i in attributes_value_ranges.Meaning.unique():
    if ('KNOWN' in str(i).upper()):
        unknown_terms.append(i)

print(unknown_terms)    

['unknown', 'unknown / no main age detectable', 'no transactions known', 'no transaction known', 'residental building buildings without actually known household', 'mixed building without actually known household or comapny ', 'company building w/o known company ', 'mixed building without actually known household ', 'mixed building without actually known company ']


In [41]:
def find_terms_in_list(llist, term):
    '''
    Searches a list for a given term
    '''

    list_of_terms = []
    for i in llist:
        if (term.upper() in str(i).upper()):
            list_of_terms.append(i)

    return list_of_terms

grob_and_fein_list = find_terms_in_list( attributes_value_ranges.Attribute, 'fein') + find_terms_in_list( attributes_value_ranges.Attribute, 'grob')
print(sorted(grob_and_fein_list))


['ALTERSKATEGORIE_GROB', 'D19_WEIN_FEINKOST_RZ', 'LP_FAMILIE_FEIN', 'LP_FAMILIE_GROB', 'LP_LEBENSPHASE_FEIN', 'LP_LEBENSPHASE_GROB', 'LP_STATUS_FEIN', 'LP_STATUS_GROB']


In [24]:
def value_range_for_attribute(df, sKey_Name):
    '''
    '''
    value_list = []
    cell_value = str(df.loc[df["Attribute"] == sKey_Name, "Value"].values[0]).split(',')
    for i in cell_value:
        value_list.append(int(i))
    
    return value_list

[-1]


In [ ]:
#azdias_sub.BIP_FLAG.unique()
list(azdias_sub.columns)

In [ ]:
pd.set_option('display.max_columns', None)
azdias_sub = azdias.head(15)
attribute_list = attributes_with_unknowns['Attribute'].tolist()
#azdias_sub

def replace_unknown_values_with_nan(df, list_of_attribs):
    '''
    '''
    attribs_not_present = []
    for attribute in list_of_attribs:
        print("replace_unknown_values_with_nan:", attribute)
        #df['CustomRating'] = df.apply(lambda x: custom_rating(x['Genre'],x['Rating']),axis=1)
        #nan_counts['percent_nan_missing'] = nan_counts[0].apply(lambda x: x*100/len(df))
        #df['col1'] = df.apply(lambda x: complex_function(x['col1']), axis=1)
        #df[attribute] = df[attribute].apply(lambda x:update_cell_to_nan_for_unknown(attribute, x))
        #df['col1'] = df.apply(lambda x: complex_function(x['col1']), axis=1)
        #df[attrib] = df[attrib].apply(replace_unkown_with_nan, args=(unkown))
        #df[attribute] = df[attribute].apply(update_cell_to_nan_for_unknown, args=(attribute))
        if attribute in df.columns:
            df[attribute] = df.apply(lambda x:update_cell_to_nan_for_unknown(attribute, x[attribute]),axis=1)
        else:
            attribs_not_present.append(attribute)

def update_cell_to_nan_for_unknown(colname, curr_value):
    print("update_cell_to_nan_for_unknown: ", colname)
    print("update_cell_to_nan_for_unknown: ", curr_value)
    #if curr_value.isin(value_range_for_attribute(attributes_with_unknowns,colname)):
    if curr_value in (value_range_for_attribute(attributes_with_unknowns,colname)):
        return np.nan
    else:
        return curr_value

#this is working
#print(update_cell_to_nan_for_unknown("AGER_TYP",-1))

#azdias["AGER_TYP"] = azdias["AGER_TYP"].apply(replace_unkown_with_nan, args=(ager_typ_unkwon))
#azdias_sub["AGER_TYP"] = azdias["AGER_TYP"].apply(update_cell_to_nan_for_unknown, args=(value_range_for_attribute("AGER_TYPE")))
replace_unknown_values_with_nan(azdias_sub, attribute_list)
#azdias_sub

In [ ]:
#jotter

#df = pd.DataFrame([[1, 2, np.nan, 4], [np.nan, np.nan, 3, 4],[1, 2, 3], [np.nan, np.nan, np.nan, 4]])
df = pd.DataFrame([[1, 2, np.nan, 4], [np.nan, np.nan, np.nan, 4],[1, 2, 3], [np.nan, 2, np.nan, 4]])
#%timeit -n 1000 df.isnull().sum(axis=1)
#x = pd.DataFrame(df.isnull().sum(axis=1))
#y = list(df.isnull().sum(axis=1))
#x = df.isnull().sum(axis=1)
#print(y)
#print(x)
#print(list(x.iloc[:,-1]))
#print(calculate_percentage_nan_per_row(df))
print(df)
print(remove_rows_with_nan_above_threshold(df,50))


In [27]:
def remove_rows_with_nan_above_threshold(df, threshold):
    '''
    '''   
    df=calculate_percentage_nan_per_row(df, True)
    df = df.drop(df[df.percent_nan > threshold].index) 
    df = df.drop('percent_nan',  axis='columns')
    
    return df

In [28]:
def calculate_percentage_nan_per_row(df, add_col_to_df=False):
    '''
    '''
    rowcount = df.shape[1] 
    nan_count = list(df.isnull().sum(axis=1))
    if add_col_to_df:
        df = df.assign(percent_nan = nan_count) 
        df['percent_nan'] = df['percent_nan'].apply(lambda x: x*100/rowcount)
        return df
    else:
        return pd.Dataframe(nan_count)
    
pd.set_option('display.max_columns', 5)
azdias_sub = azdias.head(15)
print(azdias_sub)
print()
azdias_sub = remove_rows_with_nan_above_threshold(azdias_sub, threshold = 1.9)
#zdias_sub = calculate_percentage_nan_per_row(azdias_sub, True)
print(azdias_sub)

       LNR  AGER_TYP          ...           ANREDE_KZ  ALTERSKATEGORIE_GROB
0   910215        -1          ...                   1                     2
1   910220        -1          ...                   2                     1
2   910225        -1          ...                   2                     3
3   910226         2          ...                   2                     4
4   910241        -1          ...                   1                     3
5   910244         3          ...                   2                     1
6   910248        -1          ...                   2                     2
7   910261        -1          ...                   1                     1
8   645145        -1          ...                   1                     3
9   645153        -1          ...                   2                     3
10  645165         0          ...                   2                     3
11  645169        -1          ...                   1                     2
12  612558  

In [27]:
dias_attribs.head(10)

,Information level,Attribute,Description,Additional notes
NaN,NaN,AGER_TYP,best-ager typology,in cooperation with Kantar TNS; the informatio...
NaN,Person,ALTERSKATEGORIE_GROB,age through prename analysis,modelled on millions of first name-age-referen...
NaN,NaN,ANREDE_KZ,gender,NaN
NaN,NaN,CJT_GESAMTTYP,Customer-Journey-Typology relating to the pref...,"relating to the preferred information, marketi..."
NaN,NaN,FINANZ_MINIMALIST,financial typology: low financial interest,Gfk-Typology based on a representative househo...
NaN,NaN,FINANZ_SPARER,financial typology: money saver,NaN
NaN,NaN,FINANZ_VORSORGER,financial typology: be prepared,NaN
NaN,NaN,FINANZ_ANLEGER,financial typology: investor,NaN
NaN,NaN,FINANZ_UNAUFFAELLIGER,financial typology: unremarkable,NaN
NaN,NaN,FINANZ_HAUSBAUER,financial typology: main focus is the own house,NaN


In [13]:
#find the cols with mixed data types
def findColumnsWithMixedDataTypes(df):
    '''
    '''
    columnsWithMixedDataTypes = []
    for col in df.columns:
        weird = (df[[col]].applymap(type) != df[[col]].iloc[0].apply(type)).any(axis=1)
        if len(df[weird]) > 0:
            #print(col)
            columnsWithMixedDataTypes.append(col)
    
    return columnsWithMixedDataTypes

In [6]:
#Error message on loading: DtypeWarning: Columns (18,19) have mixed types
#=> these are the only two we need to worry about
#first check do they appear in the lists above
print(azdias.columns[18:20])
print(customers.columns[18:20])

Index(['CAMEO_DEUG_2015', 'CAMEO_INTL_2015'], dtype='object')
Index(['CAMEO_DEUG_2015', 'CAMEO_INTL_2015'], dtype='object')


In [13]:
mixed_dtype_colnames = ['CAMEO_DEUG_2015', 'CAMEO_INTL_2015']
#note: CAMEO_DEUG_2015 is xxxx on the Attributes s/s

In [ ]:
#look at values in these warning cols
print(pd.value_counts(azdias.CAMEO_DEUG_2015))
print(pd.value_counts(azdias.CAMEO_INTL_2015))

print(pd.value_counts(customers.CAMEO_DEUG_2015))
print(pd.value_counts(customers.CAMEO_INTL_2015))

In [23]:
#method to update these two columns
#NB CAMEO_INTL_2015 is CAMEO_DEINTL_2015 in DIAS Attribute Values 2017 s/s 
#CAMEO_INTL_2015: object type can be set to -1
#CAMEO_DEUG_2015: object type can be set to -1
#also will cast ALL values to ints

def updateMixedDataTypeColumns(df, list_of_cols):
    '''
    '''  
    for col in list_of_cols:
        #first replace the object types with -1
        #df[col].replace({"X": -1, "XX": -1}, inplace=True)
        #first replace the object types with np.nan
        df[col].replace({"X": np.nan, "XX": np.nan}, inplace=True)
        #set Nan to -1
        df[col] = df[col].fillna(-1)
        #cast everything to int
        df[col] = df[col].astype(int)   

In [14]:
updateMixedDataTypeColumns(azdias, mixed_dtype_colnames)
updateMixedDataTypeColumns(customers, mixed_dtype_colnames)

In [15]:
def calc_percent_nan_missing(df):
    '''
    '''
    
    #get a count of the nans
    nan_counts = pd.DataFrame(df.isna().sum())
    #calculate percent_nan_missing
    nan_counts['percent_nan_missing'] = nan_counts[0].apply(lambda x: x*100/len(df))
    #reset the indexes
    nan_counts.reset_index(level=0, inplace=True)
    #set column names
    nan_counts.columns = ['feature', 'count', 'percent_nan_missing']
    #drop sum column
    nan_counts=nan_counts.drop(columns=['count'])
    
    return nan_counts

In [16]:
azdias_nan_cols = calc_percent_nan_missing(azdias)
customers_nan_cols = calc_percent_nan_missing(customers)

In [18]:
def graph_compare_nan_percentages(df1, df1_name, df2, df2_name):
    """
    """
    fig = plt.figure(figsize=(10,80))

    fig.suptitle("NaN %s")

    ax = fig.add_subplot(121)
    ax.set_title(df1_name)
    sns.barplot(y="feature", x="percent_nan_missing", data=df1[df1.percent_nan_missing>=0], ax= ax)

    ax = fig.add_subplot(122)
    ax.set_title(df2_name)
    sns.barplot(y="feature", x="percent_nan_missing", data=df2[df2.percent_nan_missing>=0], ax= ax)

    #fig.tight_layout(rect=[0, 0.03, 1, 0.975])
    fig.tight_layout()
    plt.show()

In [ ]:
graph_compare_nan_percentages(azdias_nan_cols, 'azdias', customers_nan_cols, 'customers')

In [22]:
def graph_distribution_of_num_attributes(df):
    '''
    '''
    num_attributes = df.select_dtypes(exclude='object')

    fig = plt.figure(figsize=(12,18))
    for i in range(len(num_attributes.columns)):
        fig.add_subplot(9,4,i+1)
        sns.distplot(num_attributes.iloc[:,i].dropna())
        plt.xlabel(num_attributes.columns[i])

    plt.tight_layout()
    plt.show()

In [ ]:
graph_distribution_of_num_attributes(azdias)

In [ ]:
def column_list_of_nans_below_threshold(df,threshold):
    l = []
    l = list(df.drop(df.loc[:,list((100*(df.isnull().sum()/len(df.index))>=threshold))].columns, 1).columns.values)
    print("# Columns having more than %s percent missing values:"%threshold,(df.shape[1] - len(l)))
    print("Columns:\n",list(set(list((df.columns.values))) - set(l)))
    return l

In [ ]:
#look at the numeric attributes #1
num_attributes = customers.select_dtypes(exclude='object')
num_attributes.describe()

In [ ]:
#look at the numeric attributes #2
num_attributes.isna().sum().sort_values(ascending=False).head()

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter. If you're one of the top performers, you may have the chance to be contacted by a hiring manager from Arvato or Bertelsmann for an interview!

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [ ]:
mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';')